In [ ]:
%pip install openai
%pip install langchain
%pip install langchain-community
%pip install python-dotenv
%pip install langchain-openai
%pip install opencv-python
%pip install scikit-image

In [2]:
import os

from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain_openai.chat_models import ChatOpenAI
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [23]:
def generate_text(query):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": "You are a helpful assistant."},
            {"role": "user", "content": query},
        ],
    )

    return completion.choices[0].message


def generate_image(prompt):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    return response.data[0].url


text_tool = Tool(
    name="Content Generator",
    func=generate_text,
    description="you are helpful assistant, who helps in generating the content",
)

image_tool = Tool(
    name="Image Generator",
    func=generate_image,
    description="you are helpful assistant, who helps in generating the image",
)

agent = initialize_agent(
    tools=[text_tool, image_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [5]:
text_query = "Write a short story about a robot in space."
story_response = agent.invoke(input=text_query)

In [ ]:
story_response

In [9]:
image_query = "Generate the image of the ford mustang 1969"
image_response = agent.invoke(input=image_query)

In [ ]:
image_response

In [ ]:
text_query = "I want to contact with the customer support team"
support_response = agent.invoke(input=text_query)

In [ ]:
support_response

In [30]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Chandigarh, Delhi",
                    }
                },
                "required": ["location"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "customer_support",
            "description": "trigger customer support fucntion",
            "parameters": {
                "type": "object",
                "properties": {
                    "condition": {
                        "type": "boolean",
                        "description": "e.g: True or False",
                    }
                },
                "required": ["condition"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
]


completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "i want to contact with the customer support team!"}
    ],
    tools=tools,
)


def customer_support(req):
    print(f"{req= }")
    details = {
        "url": "https://customer.flipkart.com",
        "phone no": "198748188174",
        "Should customer call you": ["yes", "no"],
    }
    return details


support_tool = Tool(
    name="Customer Support",
    func=customer_support,
    description="You are helpful customer support assistant",
)

response = completion.choices[0].message.tool_calls
function, arg = response[0].function.name, response[0].function.arguments
if function == "customer_support":
    respone = customer_support("ok")

req= 'ok'


In [47]:
respone

{'url': 'https://customer.flipkart.com',
 'phone no': '198748188174',
 'Should customer call you': ['yes', 'no']}